# Tutorial for HPS multiple A1-LCD WTARO 

In [1]:
# load packages
import numpy as np
import pandas as pd
import sys
import os
import time
try:
    import openmm as mm
    import openmm.app as app
    import openmm.unit as unit
except ImportError:
    import simtk.openmm as mm
    import simtk.openmm.app as app
    import simtk.unit as unit

import mdtraj
try:
    import nglview
except ImportError:
    print('Please install nglview to visualize molecules in the jupyter notebooks.')

sys.path.append('../../')
from openabc.forcefields.parsers import HPSParser
from openabc.forcefields import HPSModel
from openabc.utils.helper_functions import build_straight_CA_chain, write_pdb
from openabc.utils.insert import insert_molecules

# set simulation platform
platform_name = 'CPU'
#platform_name = 'OpenCL'

Parse a single A1LCD WT. The sequence length is 137.

In [2]:
sequence = 'GSMASASSSQRGRSGSGNFGGGRGGGFGGNDNFGRGGNFSGRGGFGGSRGGGGYGGSGDGYNGFGNDGSNFGGGGSYNDFGNYNNQSSNFGPMKGGNFGGRSSGGSGGGGQYFAKPRNQGGYGGSSSSSSYGSGRRF'
ca_pdb = 'init_A1LCD_WT_CA.pdb'
ca_atoms = build_straight_CA_chain(sequence, r0=0.38)
write_pdb(ca_atoms, ca_pdb)
protein_parser = HPSParser(ca_pdb)

Parse molecule with default settings.


Prepare the initial configuration.

In [3]:
# insert molecules into the simulation box randomly
n_mol = 200
if not os.path.exists('start.pdb'):
    insert_molecules(ca_pdb, 'start.pdb', n_mol, box=[100, 100, 100])
else:
    # delete and create a new start.pdb
    # otherwise, there will be an error for init_coord in the next step
    os.remove('start.pdb')
    insert_molecules(ca_pdb, 'start.pdb', n_mol, box=[100, 100, 100])


Successfully inserted 200 molecules.


Use the Urry scale optimal parameter ($\mu=1$ and $\Delta=0.08$) and run the simulation. 

In [4]:
protein = HPSModel()
for i in range(n_mol):
    protein.append_mol(protein_parser)
top = app.PDBFile('start.pdb').getTopology()
init_coord = app.PDBFile('start.pdb').getPositions()
print('size of init_coord:', len(init_coord))
protein.create_system(top, box_a=100, box_b=100, box_c=100)
protein.add_protein_bonds(force_group=1)
protein.add_contacts('Urry', mu=1, delta=0.08, force_group=2)
protein.add_dh_elec(force_group=3)
temperature = 274*unit.kelvin
friction_coeff = 0.01/unit.picosecond # use smaller friction coefficient to accelerate dynamics
timestep = 10*unit.femtosecond
integrator = mm.LangevinMiddleIntegrator(temperature, friction_coeff, timestep)
protein.set_simulation(integrator, platform_name, init_coord=init_coord)
protein.simulation.minimizeEnergy()
output_interval = 100
output_dcd = 'output_multi_A1LCD_WT.dcd'
protein.add_reporters(output_interval, output_dcd)
protein.simulation.context.setVelocitiesToTemperature(temperature)

# run simulation
start = time.time()
totaL_steps = 10000
protein.simulation.step(totaL_steps)
end = time.time()
print('Total time for %d steps: %.2f s' % (totaL_steps, end-start))

size of init_coord: 27400
Add protein bonds.
Add nonbonded contacts.
Use Urry hydropathy scale.
Scale factor mu = 1 and shift delta = 0.08.
Add Debye-Huckel electrostatic interactions.
Set Debye length as 1 nm.
Set water dielectric as 80.0.
Use platform: CPU
#"Step","Time (ps)","Potential Energy (kJ/mole)","Kinetic Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
100,1.0000000000000007,26468.60933041258,61926.294391858086,88394.90372227067,181.22367642560994,0
200,2.0000000000000013,30025.58971202999,58964.6455263951,88990.23523842508,172.55658434539168,170
300,2.99999999999998,27015.005134215986,62708.73572049161,89723.7408547076,183.51344518304668,172
400,3.9999999999999587,25582.110235295848,64718.47634990996,90300.5865852058,189.39483351900043,173
500,4.999999999999938,23279.55009774895,67574.72797284022,90854.27807058918,197.75348673710891,173
600,5.9999999999999165,21595.61772013994,69818.15585190963,91413.77357204957,204.31874713308895,173
700,6.9999

In [5]:
# visualize trajectory
traj = mdtraj.load_dcd('output_multi_A1LCD_WT.dcd', top='start.pdb')
traj.xyz -= np.mean(traj.xyz, axis=1, keepdims=True) # realign to the origin
view = nglview.show_mdtraj(traj)
view

NGLWidget(max_frame=99)